In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import (Conv2D,LeakyReLU,Dense,Flatten,ZeroPadding2D,
                                     Dropout,BatchNormalization,UpSampling2D,
                                     GlobalMaxPooling2D,Reshape,Conv2DTranspose)
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, callbacks


In [5]:
epoch=1000
if epoch %500 == 0:
    print('Epoch {}'.format(epoch))


Epoch 1000


In [3]:
X_train = np.load('x_train.npy')
X_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


In [62]:
X_train = X_train.astype('float32') /255
X_test = X_test /255

In [5]:
latent_dim= 128
batch_size = 64

In [63]:

all_images = np.reshape(X_train,(-1,28,28,1))
dataset = tf.data.Dataset.from_tensor_slices(all_images)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(3)

2023-03-27 15:33:38.428729: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


In [52]:
class Create_Models():
    def __init__(self,latent_dim):
        self.latent_dim= latent_dim
    
    def build_disc(self):
        model = Sequential()
        model.add(ZeroPadding2D(padding=(1, 1), input_shape=(28, 28,1)))
        model.add(Conv2D(32,3))
        model.add(ReLU())
        model.add(Dropout(0.3))
            
        model.add(Conv2D(64,3,padding='same'))
        model.add(ReLU())
        model.add(MaxPool2D(pool_size=2, strides=2))

        
        model.add(Conv2D(128,3,strides=1,padding='same'))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Dropout(0.5))
        
        model.add(Conv2D(256,3,strides=1,padding='same'))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(Dropout(0.5))

        model.add(Conv2D(128,3,strides=1,padding='same'))
        model.add(BatchNormalization())
        model.add(ReLU())
        
        model.add(Flatten())
        model.add(Dropout(0.5))
        
        model.add(Dense(32, activation='relu')) 
        model.add(Dense(1, activation='sigmoid')) 
        
        return model
    
    def build_gen(self):
        model = Sequential()
        
        model.add(Dense(7*7*128,input_dim =self.latent_dim))
        model.add(ReLU())
        model.add(Reshape((7,7,128)))

        model.add(Conv2DTranspose(128,4,strides=2,padding='same'))
        model.add(ReLU())
        model.add(BatchNormalization())

        
        model.add(Conv2DTranspose(64,4,strides=1,padding='same'))
        model.add(ReLU())
        model.add(BatchNormalization())
        

        model.add(Conv2DTranspose(32,4,strides=2,padding='same'))
        model.add(ReLU())
        model.add(BatchNormalization())
        
        
    
        model.add(Conv2D(1,7,padding='same',activation='tanh'))
        return model

In [53]:
creat_model = Create_Models(latent_dim)
generator= creat_model.build_gen()
discriminator= creat_model.build_disc()

In [54]:
generator.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 6272)              809088    
                                                                 
 re_lu_67 (ReLU)             (None, 6272)              0         
                                                                 
 reshape_6 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose_18 (Conv2D  (None, 14, 14, 128)      262272    
 Transpose)                                                      
                                                                 
 re_lu_68 (ReLU)             (None, 14, 14, 128)       0         
                                                                 
 batch_normalization_43 (Bat  (None, 14, 14, 128)      512       
 chNormalization)                                    

In [68]:
class DCGAN(Model):
    def __init__(self, discriminator, generator, latent_dim,*args,**kwargs):
        super(DCGAN, self).__init__(*args,**kwargs)
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        
    def compile(self, d_opt, g_opt,loss_fn,*args,**kwargs):
        super(DCGAN, self).compile(*args,**kwargs)
        self.d_opt = d_opt
        self.g_opt =g_opt
        self.loss_fn = loss_fn
        
    def train_step(self, real_images):
        batch_size= tf.shape(real_images)[0]
        random_latent_vects = tf.random.normal(shape=(batch_size,self.latent_dim))
        gen_images = self.generator(random_latent_vects)
        combind_images= tf.concat([gen_images, real_images],axis=0)
        
        
        labels = tf.concat([tf.ones((batch_size,1)), tf.zeros((batch_size,1))],axis=0)
        labels += 0.04* tf.random.uniform(tf.shape(labels))
        
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combind_images)
            d_loss = self.loss_fn(labels, predictions)
            
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_opt.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )
        
        
        random_latent_vects = tf.random.normal(shape=(batch_size,self.latent_dim))
        fake_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vects))
            g_loss = self.loss_fn(fake_labels,predictions)
            

        grads = tape.gradient(g_loss,self.generator.trainable_weights)
        self.g_opt.apply_gradients(
            zip(grads,self.generator.trainable_weights)
        )
        
        
        return {"d_loss":d_loss,"g_loss":g_loss}


In [69]:
class DCGAN_Monitor(callbacks.Callback):
    def __init__(self,latent_dim):
        self.latent_dim = latent_dim
    
    def on_epoch_end(self, epoch, logs=None):
        random_latent_vects =tf.random.normal(shape=(16, self.latent_dim))
        gen_images = self.model.generator(random_latent_vects)
        
        plt.close('all')
        
        fix, axs = plt.subplots(4,4, figsize(4,4), sharey= True, sharex=True)
        
        c=0 
        for i in range(4):
            for j in range(4):
                axs[i,j].imshow(gen_images[c,:,:,0],cmap='gray')
                axs[i,j].axis('off')
                c+=1
                
        fig.show()
                

In [ ]:
epochs=1
dcgan = DCGAN(discriminator=discriminator,generator=generator,latent_dim=latent_dim)
dcgan.compile(
    d_opt=tf.keras.optimizers.Adam(learning_rate=0.0002,beta_1=0.5),
    g_opt=tf.keras.optimizers.Adam(learning_rate=0.0002,beta_1=0.5),
    loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)
dcgan.fit(dataset,epochs=epochs,callbacks=[DCGAN_Monitor(latent_dim=latent_dim)])

2023-03-27 15:35:03.252615: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18464768 exceeds 10% of free system memory.


544/938 [================>.............] - ETA: 15:51 - d_loss: 0.6923 - g_loss: 0.7142